In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.video import *

base = Overlay("/home/xilinx/jupyter_notebooks/quwu/base_wrapper.bit")
base.download()

In [13]:
import cv2

#设置hdmi输出分辨率为1280*720
Mode = VideoMode(1280,720,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
#设置usb摄像头输入分辨率为1280*720
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, 1280);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, 720);
print("capture device is open: " + str(videoIn.isOpened()))

capture device is open: True


In [14]:
#开启usb去雾
import numpy as np

fog = base.video.hdmi_out.img_defog_0
#大气光参数为230
fog.write(0x10,230)

while(1):
    ret, frame_vga = videoIn.read()

    if (ret):
        outframe = hdmi_out.newframe()
        outframe[:] = frame_vga
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Error while reading from camera.")
        
#停止输出请先停止此段程序

Exception ignored in: <bound method ContiguousArray.__del__ of ContiguousArray([[[103,  84,  87],
                  [102,  83,  86],
                  [ 99,  82,  86],
                  ..., 
                  [186, 178, 178],
                  [188, 180, 180],
                  [189, 181, 181]],

                 [[102,  82,  87],
                  [101,  81,  86],
                  [ 98,  80,  87],
                  ..., 
                  [186, 178, 178],
                  [187, 179, 179],
                  [188, 180, 180]],

                 [[ 99,  81,  88],
                  [ 98,  80,  87],
                  [ 94,  77,  86],
                  ..., 
                  [188, 180, 180],
                  [188, 180, 180],
                  [188, 180, 180]],

                 ..., 
                 [[150, 120, 131],
                  [151, 121, 132],
                  [151, 124, 134],
                  ..., 
                  [ 50,  42,  49],
                  [ 51,  40,  50],
       

KeyboardInterrupt: 

In [15]:
#释放hdmi资源和usb摄像头
videoIn.release()
hdmi_out.stop()
del hdmi_out

In [16]:
#此段程序为hdmi输入去雾
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
HDMI_Mode = VideoMode(1280,720,24)
hdmi_in.configure()
a = hdmi_in.mode
a.width = 1280
a.height=720
hdmi_out.configure(a)
fog = base.video.hdmi_out.img_defog_0
#大气光参数为230
fog.write(0x10,230)


In [21]:
#开启hdmi去雾
hdmi_in.start()
hdmi_out.start()
hdmi_in.tie(hdmi_out)

In [22]:
#停止hdmi去雾
hdmi_in.stop()
hdmi_out.stop()